# Assignment 3
###### Team Members: 
Katelyn Vincent (8:30am)  
Jenny Robinson (10:30am)  
Jordan Pflum (10:30am)  
Bryant Leal (10:30am)  
Kessiena Ofunrein (8:30am)

## Google API

In [ ]:
#pip install --user google-cloud
#pip install --user google-cloud-vision

import xlrd
from google.cloud import vision
import os
import io
import pandas as pd
from google.cloud import vision_v1 as vision

In [ ]:
image_uri = 'https://instagram.fhou1-2.fna.fbcdn.net/v/t51.2885-15/e35/p1080x1080/118654844_826359864598015_7262611505862937905_n.jpg?_nc_ht=instagram.fhou1-2.fna.fbcdn.net&_nc_cat=107&_nc_ohc=uOrQAZ098CcAX-87vXb&tp=19&oh=a3c7c4a4fe5862f1cab0a7b5824de81e&oe=5FBC7212'
client = vision.ImageAnnotatorClient()
image = vision.types.Image()
image.source.image_uri = image_uri

response = client.label_detection(image=image)

In [ ]:
print('Labels (and confidence score):')
print('=' * 30)
for label in response.label_annotations:
    print(label.description)

## Task A

Create a metric for engagement by using a weighted sum of # likes and # comments. However, first normalize # likes and # comments such that they both have values between 0 and 1. You can scale the # likes by dividing by the maximum # likes (for a post) in your data and do the same for # comments, so that # likes and comments will be in the range [0,1]. Now create an engagement score = .4*# likes (normalized) + .6*# comments (normalized). Define High (1) and Low (0) engagement based on whether the engagement score is above or below the median value.  

In [209]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer

In [210]:
# calculate engagement score
df = pd.read_csv('zarafake.csv')
df['norm_likes'] = df['likes']/df['likes'].max()
df['norm_comments'] = df['comments']/df['comments'].max()
df['engagement'] = (.4*df['norm_likes']) + (.6*df['norm_comments'])
df['engagement_binary'] = np.where(df['engagement'] >= df['engagement'].median(), 1, 0)
df.head()

,url,caption,comments,likes,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9,label_10,norm_likes,norm_comments,engagement,engagement_binary
0,https://instagram.fhou1-2.fna.fbcdn.net/v/t51....,The female gaze. Lingerie collection\nPhotogra...,246,42971,photography,black,white,fashion,turquoise,purse,body,woman,outerwear,textiles,0.146610,0.075786,0.104115,0
1,https://instagram.fhou1-2.fna.fbcdn.net/v/t51....,The female gaze. Lingerie collection\nPhotogra...,207,73761,photography,black,white,fashion,turquoise,purse,body,woman,outerwear,textiles,0.251661,0.063771,0.138927,1
2,https://instagram.fhou1-2.fna.fbcdn.net/v/t51....,The female gaze. Lingerie collection\nPhotogra...,121,41218,red,black,photography,blue,yellow,purse,body,woman,outerwear,textiles,0.140629,0.037277,0.078618,0
3,https://instagram.fhou1-2.fna.fbcdn.net/v/t51....,FW20 Campaign. Kids Collection\nCreative Direc...,426,74112,red,black,photography,blue,yellow,purse,body,woman,outerwear,textiles,0.252858,0.131238,0.179886,1
4,https://instagram.fhou1-2.fna.fbcdn.net/v/t51....,FW20 Campaign. Kids Collection\nCreative Direc...,133,38872,photography,black,white,fashion,turquoise,purse,body,woman,outerwear,textiles,0.132625,0.040974,0.077634,0


## Task B

Using TF-IDF scores, run a logistic regression with engagement (binary) as the dependent variable, and the image labels as independent variables. What is the accuracy (show the confusion matrix)?
What accuracy do you get by using the post caption words as the independent variables instead of image labels? Finally, what accuracy do you get by combining the image labels and post captions and using them as independent variables? What can you conclude from your analysis?

In [211]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer 

In [212]:
# create 'document' for each group of labels
cols = ['label_1', 'label_2', 'label_3', 'label_4', 'label_5', 'label_6', 'label_7', 'label_8', 'label_9', 'label_10']
df['document'] = df[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
docs = df['document'].to_list()
docs[:5]

['photography black white fashion turquoise purse body woman outerwear textiles',
 'photography black white fashion turquoise purse body woman outerwear textiles',
 'red black photography blue yellow purse body woman outerwear textiles',
 'red black photography blue yellow purse body woman outerwear textiles',
 'photography black white fashion turquoise purse body woman outerwear textiles']

In [213]:
# settings that you use for count vectorizer will go here 
tfidf_vectorizer=TfidfVectorizer(use_idf=True) 
 
# just send in all your docs here 
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(docs)

# get the first vector out (for the first document) 
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0] 
 
# place tf-idf values in a pandas data frame 
idf_scores = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
idf_scores.sort_values(by=["tfidf"],ascending=False)

,tfidf
fashion,0.399067
turquoise,0.399067
white,0.399067
black,0.273140
body,0.273140
outerwear,0.273140
photography,0.273140
purse,0.273140
textiles,0.273140
woman,0.273140
